# Move and trim files recorded on Feb 16th, Performance

Check and clear 

Plot and review measurements from equivital sensors as output by the Equivital Manager app.
(Every program produces different formated files and it's a pain.)

In [17]:
import sys
import os
import shutil
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

## Definitions for data extraction

In [18]:
%load_ext autoreload
%autoreload 1
%aimport qex

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
%reload_ext autoreload

In [20]:
seperator = str('\\')
seperator

'\\'

# Scan for DATA files

All recordinds exported by Qiosk produce four CSV files and one SEM file. One of the CSVs is a DATA file that contains metadata and some essential signal quality checks on 15 s intervals.

We can ID instances of signal recordings from statistics taken out of the DATA file, so that gets extracted first

In [21]:
# location of the files when exported by QIOSK
path = "C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\"#Extracted SEM Data\\" #"../Equivital/Equivital Manager Wizard/Extracted SEM Data/"
#os.listdir(path)

df_datafiles = qex.qiosk_recordings(path,'Present',seperator)
df_datafiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,BR605,5022202,230214,0,DATA-BR605-5022202-23021400.CSV,csv,26127,2023-02-14 12:17:29+00:00,2023-02-14 13:36:59+00:00,...,[BR605],5022202.0,BR605,NaN,46.0,97.0,0.0,1.0,Stationary,Side
1,DATA,BR606,5022203,230214,101,DATA-BR606-5022203-23021400_0.CSV,csv,25515,2023-02-14 12:19:12+00:00,2023-02-14 13:37:12+00:00,...,[BR606],5022203.0,BR606,NaN,20.0,98.0,0.0,1.0,Stationary,Prone
2,DATA,VN208,5022076,230214,101,DATA-VN208-5022076-23021400_0.CSV,csv,28947,2023-02-14 12:20:05+00:00,2023-02-14 13:48:35+00:00,...,[VN208],5022076.0,VN208,NaN,12.0,38.0,0.0,1.0,Stationary,Prone
3,DATA,BR607,5022204,230214,0,DATA-BR607-5022204-23021400.CSV,csv,24318,2023-02-14 12:24:26+00:00,2023-02-14 13:37:11+00:00,...,[BR607],5022204.0,BR607,NaN,106.0,38.0,0.0,1.0,Stationary,Unknown
4,DATA,VN210,5022098,230214,101,DATA-VN210-5022098-23021400_0.CSV,csv,25254,2023-02-14 12:31:54+00:00,2023-02-14 13:48:39+00:00,...,[VN210],5022098.0,VN210,NaN,12.0,99.0,0.0,1.0,Stationary,Prone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,DATA,VN201,5022050,230216,0,DATA-VN201-5022050-23021600.CSV,csv,112184,2023-02-16 07:13:35+00:00,2023-02-16 12:51:50+00:00,...,[VN201],5022050.0,VN201,NaN,72.0,98.0,0.0,0.0,Stationary,Upright
70,DATA,LS405,5022143,230216,101,DATA-LS405-5022143-23021600_0.CSV,csv,112477,2023-02-16 07:13:42+00:00,2023-02-16 12:53:12+00:00,...,[LS405],5022143.0,LS405,NaN,76.0,98.0,0.0,0.0,Stationary,Upright
71,DATA,VN202,5022051,230216,0,DATA-VN202-5022051-23021600.CSV,csv,111633,2023-02-16 07:14:30+00:00,2023-02-16 12:51:45+00:00,...,[VN202],5022051.0,VN202,NaN,87.0,99.0,0.0,0.0,Stationary,Upright
72,DATA,VN205,5022060,230216,0,DATA-VN205-5022060-23021600.CSV,csv,110192,2023-02-16 07:14:31+00:00,2023-02-16 12:48:31+00:00,...,[VN205],5022060.0,VN205,NaN,66.0,98.0,0.0,0.0,Stationary,Upright


In [6]:
print('Number of DATA files: '+ str(len(df_datafiles)))
# key categories to structure of Qiosk output DATA files
print(df_datafiles['ID'].unique())
print(df_datafiles['DevName'].unique())
print(df_datafiles['Date'].unique())

Number of DATA files: 265
[5022281 5022050 5022247 5022051 5022245 5022057 5022244 5022241 5022058
 5022240 5022238 5022073 5023926 5022062 5022149 5022031 3420426 5022060
 5022152 5022154 5022151 5022256 3420427 3420415 3420456 5022131 5022127
 5022126 3420755 5022117 5022026 5022299 5022010 5022009 5022027 5022100
 5022006 5022101 5022002 5022104 5022110 5022143 5022142 5022163 5022202
 5022198 5022076 5022193 5022164 5022184 5022179 5022297 5022166 5022292
 5022173 5022285 5022176 5022203 5022161 5022204 5022098 5022077 5022138
 5022222 5022155 5022234 5022156 5022139 5022141 5022207]
['Bluetooth01' 'VN201' 'PC706' 'VN202' 'PC705' 'VN203' 'PC704' 'PC703'
 'VN204' 'PC702' 'PC701' 'VN207' 'LS406' 'VN206' 'LS407' 'VN112' 'VN102'
 'VN205' 'LS409' 'LS410' 'LS408' 'PC707' 'VN103' 'VN101' 'VN104' 'VA308'
 'VA307' 'VA306' 'VN105' 'VA305' 'VN110' 'SPARE' 'VN109' 'VN108' 'VN111'
 'VA301' 'VN107' 'VA302' 'VN106' 'VA303' 'VA304' 'LS405' 'LS404' 'WW504'
 'BR605' 'BR604' 'VN208' 'BR603' 'WW505' '

# discard tiny


In [22]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230216
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


In [23]:
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
20,DATA,WW503,5022161,230216,0,DATA-WW503-5022161-23021600.CSV,csv,115705,2023-02-16 07:03:50+00:00,2023-02-16 12:52:50+00:00,...,[WW503],5022161.0,WW503,NaN,90.0,98.0,0.0,0.0,Stationary,Upright
21,DATA,LS404,5022142,230216,0,DATA-LS404-5022142-23021600.CSV,csv,115318,2023-02-16 07:03:50+00:00,2023-02-16 12:53:07+00:00,...,[LS404],5022142.0,LS404,NaN,75.0,98.0,0.0,0.0,Stationary,Upright
22,DATA,VA304,5022110,230216,0,DATA-VA304-5022110-23021600.CSV,csv,116243,2023-02-16 07:03:53+00:00,2023-02-16 12:54:53+00:00,...,[VA304],5022110.0,VA304,NaN,62.0,98.0,0.0,0.0,Stationary,Upright
23,DATA,BR601,5022179,230216,0,DATA-BR601-5022179-23021600.CSV,csv,119707,2023-02-16 07:03:53+00:00,2023-02-16 13:04:38+00:00,...,[BR601],5022179.0,BR601,NaN,65.0,97.0,0.0,0.0,Stationary,Upright
24,DATA,PC705,5022245,230216,0,DATA-PC705-5022245-23021600.CSV,csv,115582,2023-02-16 07:03:55+00:00,2023-02-16 12:52:55+00:00,...,[PC705],5022245.0,PC705,NaN,83.0,97.0,0.0,1.0,Stationary,Upright
25,DATA,BR603,5022193,230216,0,DATA-BR603-5022193-23021600.CSV,csv,119370,2023-02-16 07:03:56+00:00,2023-02-16 13:05:11+00:00,...,[BR603],5022193.0,BR603,NaN,0.0,98.0,0.0,0.0,Stationary,Upright
26,DATA,PC707,5022256,230216,0,DATA-PC707-5022256-23021600.CSV,csv,113508,2023-02-16 07:04:44+00:00,2023-02-16 12:47:14+00:00,...,[PC707],5022256.0,PC707,NaN,0.0,98.0,0.0,0.0,Stationary,Upright
27,DATA,WW502,5022156,230216,0,DATA-WW502-5022156-23021600.CSV,csv,114897,2023-02-16 07:04:44+00:00,2023-02-16 12:52:44+00:00,...,[WW502],5022156.0,WW502,NaN,72.0,98.0,0.0,1.0,Stationary,Upright
28,DATA,VA301,5022100,230216,0,DATA-VA301-5022100-23021600.CSV,csv,115385,2023-02-16 07:04:47+00:00,2023-02-16 12:52:32+00:00,...,[VA301],5022100.0,VA301,NaN,84.0,98.0,0.0,0.0,Stationary,Upright
29,DATA,PC701,5022238,230216,0,DATA-PC701-5022238-23021600.CSV,csv,115881,2023-02-16 07:04:48+00:00,2023-02-16 12:53:03+00:00,...,[PC701],5022238.0,PC701,NaN,78.0,39.0,0.0,1.0,Stationary,Upright


## Define set of recordings to move

We sort files by day and time recording

In [12]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230216
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


Additionally select according to start and stop recording times.

In [13]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-16 9:34:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']< t,:] 

In [14]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-16 07:00:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']> t,:] 

In [15]:
# recordings ended after a certain time
t = pd.to_datetime('2023-02-16 10:30:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecEnd']> t,:] 

In [16]:
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,FullLoc,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
209,DATA,LS404,5022142,230216,0,DATA-LS404-5022142-23021600.CSV,csv,115318,2023-02-16 07:03:50+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,[LS404],5022142.0,LS404,NaN,75.0,98.0,0.0,0.0,Stationary,Upright
210,DATA,WW503,5022161,230216,0,DATA-WW503-5022161-23021600.CSV,csv,115705,2023-02-16 07:03:50+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,[WW503],5022161.0,WW503,NaN,90.0,98.0,0.0,0.0,Stationary,Upright
211,DATA,VA304,5022110,230216,0,DATA-VA304-5022110-23021600.CSV,csv,116243,2023-02-16 07:03:53+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,[VA304],5022110.0,VA304,NaN,62.0,98.0,0.0,0.0,Stationary,Upright
212,DATA,BR601,5022179,230216,0,DATA-BR601-5022179-23021600.CSV,csv,119707,2023-02-16 07:03:53+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,[BR601],5022179.0,BR601,NaN,65.0,97.0,0.0,0.0,Stationary,Upright
213,DATA,PC705,5022245,230216,0,DATA-PC705-5022245-23021600.CSV,csv,115582,2023-02-16 07:03:55+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,[PC705],5022245.0,PC705,NaN,83.0,97.0,0.0,1.0,Stationary,Upright
214,DATA,BR603,5022193,230216,0,DATA-BR603-5022193-23021600.CSV,csv,119370,2023-02-16 07:03:56+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,[BR603],5022193.0,BR603,NaN,0.0,98.0,0.0,0.0,Stationary,Upright
215,DATA,PC707,5022256,230216,0,DATA-PC707-5022256-23021600.CSV,csv,113508,2023-02-16 07:04:44+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,[PC707],5022256.0,PC707,NaN,0.0,98.0,0.0,0.0,Stationary,Upright
216,DATA,WW502,5022156,230216,0,DATA-WW502-5022156-23021600.CSV,csv,114897,2023-02-16 07:04:44+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,[WW502],5022156.0,WW502,NaN,72.0,98.0,0.0,1.0,Stationary,Upright
217,DATA,VA301,5022100,230216,0,DATA-VA301-5022100-23021600.CSV,csv,115385,2023-02-16 07:04:47+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,[VA301],5022100.0,VA301,NaN,84.0,98.0,0.0,0.0,Stationary,Upright
218,DATA,PC701,5022238,230216,0,DATA-PC701-5022238-23021600.CSV,csv,115881,2023-02-16 07:04:48+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,[PC701],5022238.0,PC701,NaN,78.0,39.0,0.0,1.0,Stationary,Upright


In [29]:
# show subset of files
s_files.iloc[0,:]

Signal                                                            DATA
DevName                                                          WW503
ID                                                             5022161
Date                                                            230216
Session                                                              0
FileName                               DATA-WW503-5022161-23021600.CSV
FileType                                                           csv
FileSize                                                        115705
RecStart                                     2023-02-16 07:03:50+00:00
RecEnd                                       2023-02-16 12:52:50+00:00
Duration                                                       20940.0
BatteryStart                                                      4088
BatteryEnd                                                        3959
BatteryChange(mV)                                                 -129
FullLo

In [15]:
s_files.max()

Signal                                                            DATA
DevName                                                          WW508
ID                                                             5023926
Date                                                            230215
Session                                                            102
FileName                               DATA-WW508-5022176-23021501.CSV
FileType                                                           csv
FileSize                                                        203597
RecStart                                     2023-02-15 10:13:51+00:00
FullLoc              C:\Users\Public\Documents\Equivital\Equivital ...
RecEnd                                       2023-02-15 16:50:18+00:00
Duration                                                       37230.0
BatteryStart                                                    4170.0
BatteryEnd                                                      4158.0
Batter

In [16]:
s_files.min()

Signal                                                            DATA
DevName                                                          BR601
ID                                                             3420415
Date                                                            230215
Session                                                              0
FileName                               DATA-BR601-5022179-23021500.CSV
FileType                                                           csv
FileSize                                                           217
RecStart                                     2023-02-15 06:27:44+00:00
FullLoc              C:\Users\Public\Documents\Equivital\Equivital ...
RecEnd                                       2023-02-15 07:27:41+00:00
Duration                                                           0.0
BatteryStart                                                    4082.0
BatteryEnd                                                      3795.0
Batter

These recordings can be checked in View_Equivital_Qiosk_Export.

Now we move them

# Move selected files to suitable project folder

In [25]:
projectsFolder =  'D:\\' # where qiosk files are moved to after export
print(projectsFolder) # where the qiosk files are initially saved by the Equivital Qiosk program

D:\


In [26]:
# set project folder name
projectName = '230216_Lydo_Concerts'
projectPath = projectsFolder + projectName + '\\'

In [27]:
discardPath = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\'

### Create folders for project to move files to
Check target location for data files to keep and to remove

In [28]:
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')

# Set Interval to keep in CSV

We will cut the the CSV files to save on space and presever privacy. 
In this case, we are only taking the 

In [30]:
t1 = pd.to_datetime('2023-02-16 09:15:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]
t2 = pd.to_datetime('2023-02-16 12:30:00.00+0100')
#t2 = t1+pd.to_timedelta(240,'s')
[t1,t2]

[Timestamp('2023-02-16 09:15:00+0100', tz='pytz.FixedOffset(60)'),
 Timestamp('2023-02-16 12:30:00+0100', tz='pytz.FixedOffset(60)')]

In [31]:
(t2-t1).total_seconds()

11700.0

In [54]:
# test cutting
tic = time.time()

dataLoc = s_files.iloc[0,14]
matched = qex.matched_files(dataLoc,path,seperator) # outputs locations of csv 
for fi in matched:
    if not fi.lower().endswith('.sem'):
        fp = fi.split(seperator)
        fn = fp[-1].split('.')
        fn_new = fn[0]+'-trimmed.csv'
        print(fn_new)
        V = qex.cut_by_time(fi,t1,t2)
        print(V.columns)
        V.to_csv(projectPath + 'CSV\\' + fn_new)
        print(time.time()-tic)
    if eq_file_loc.lower().endswith('sem'):
        if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
            os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
        out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
        os.rename(eq_file_loc,out_f)
        print(time.time()-tic)

DATA-VN201-5022050-23021400-trimmed.csv
Index(['SENSOR ID', 'SUBJECT ID', 'SUBJECT AGE', 'HR(BPM)', 'HRC(%)',
       'BELT OFF', 'LEAD OFF', 'MOTION', 'BODY POSITION', 'BATTERY(mV)',
       'TEMPERATURE', 'BUTTON PRESS'],
      dtype='object')
0.4191780090332031
ECG-VN201-5022050-23021400-trimmed.csv
Index(['Lead 1', 'Lead 2'], dtype='object')
47.62759590148926
FASTACC-VN201-5022050-23021400-trimmed.csv
Index(['Vert Accelerometer', 'Lat Accelerometer', 'Long Accelerometer'], dtype='object')
90.35752868652344
RESPACC-VN201-5022050-23021400-trimmed.csv
Index(['Breathing'], dtype='object')
94.61357927322388
RR-VN201-5022050-23021400-trimmed.csv
Index(['Interbeat Interval (MS)'], dtype='object')
94.8211452960968


# Full Cut Save Move Discard

In [ ]:
tac = time.time()

for i,row in s_files.iterrows():
    tic = time.time()
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path,seperator)  # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                fn = fileName.split('.')
                fn_new = fn[0]+'-cut.csv'
                out_f = projectPath + 'CSV\\' + fn_new
                print(out_f)
                V = qex.cut_by_time(fi,t1,t2)
                V.to_csv(out_f) # write the trimmed csv to the external harddisk
                
                dout_f = discardPath + 'CSV\\' + fileName
                shutil.move(fi,dout_f) # move the exported csv files to the discard folder
                print(time.time()-tic)
        if fileName.lower().endswith('sem'):
            devName = row['DevName']
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            shutil.move(fi,out_f)
            print(time.time()-tic)
            
    print([time.time()-tic, time.time()-tac])

dfiles = qex.qiosk_recordings(projectPath,projectName,seperator)
print(projectPath)
print(projectName)
dfiles

D:\230216_Lydo_Concerts\CSV\DATA-WW503-5022161-23021600-cut.csv
0.3651895523071289
D:\230216_Lydo_Concerts\CSV\ECG-WW503-5022161-23021600-cut.csv
22.74461054801941
D:\230216_Lydo_Concerts\CSV\FASTACC-WW503-5022161-23021600-cut.csv
47.57980251312256
D:\230216_Lydo_Concerts\CSV\RESPACC-WW503-5022161-23021600-cut.csv
49.7533392906189
D:\230216_Lydo_Concerts\CSV\RR-WW503-5022161-23021600-cut.csv
49.988511085510254
51.00357532501221
[51.00357532501221, 51.00357532501221]
D:\230216_Lydo_Concerts\CSV\DATA-LS404-5022142-23021600-cut.csv
0.0368809700012207
D:\230216_Lydo_Concerts\CSV\ECG-LS404-5022142-23021600-cut.csv
36.274452209472656
D:\230216_Lydo_Concerts\CSV\FASTACC-LS404-5022142-23021600-cut.csv
59.91215944290161
D:\230216_Lydo_Concerts\CSV\RESPACC-LS404-5022142-23021600-cut.csv
63.1763813495636
D:\230216_Lydo_Concerts\CSV\RR-LS404-5022142-23021600-cut.csv
63.38073205947876
64.2818295955658
[64.28283309936523, 115.28640842437744]
D:\230216_Lydo_Concerts\CSV\DATA-VA304-5022110-23021600-cu

70.09391784667969
D:\230216_Lydo_Concerts\CSV\RESPACC-VA306-5022126-23021600-cut.csv
73.50722861289978
D:\230216_Lydo_Concerts\CSV\RR-VA306-5022126-23021600-cut.csv
73.70744967460632
74.56951093673706
[74.56951093673706, 1123.2398657798767]
D:\230216_Lydo_Concerts\CSV\DATA-LS407-5022149-23021600-cut.csv
0.03288984298706055
D:\230216_Lydo_Concerts\CSV\ECG-LS407-5022149-23021600-cut.csv
34.819037199020386
D:\230216_Lydo_Concerts\CSV\FASTACC-LS407-5022149-23021600-cut.csv
66.29759979248047
D:\230216_Lydo_Concerts\CSV\RESPACC-LS407-5022149-23021600-cut.csv
69.2168698310852
D:\230216_Lydo_Concerts\CSV\RR-LS407-5022149-23021600-cut.csv
69.45706701278687
70.41397953033447
[70.41397953033447, 1193.6548418998718]
D:\230216_Lydo_Concerts\CSV\DATA-VN108-5022009-23021600-cut.csv
0.05083012580871582
D:\230216_Lydo_Concerts\CSV\ECG-VN108-5022009-23021600-cut.csv
38.549277544021606
D:\230216_Lydo_Concerts\CSV\FASTACC-VN108-5022009-23021600-cut.csv
64.38209462165833
D:\230216_Lydo_Concerts\CSV\RESPACC

58.952089071273804
[58.95308589935303, 2223.364889860153]
D:\230216_Lydo_Concerts\CSV\DATA-VN206-5022062-23021600-cut.csv
0.0378727912902832
D:\230216_Lydo_Concerts\CSV\ECG-VN206-5022062-23021600-cut.csv
32.99208164215088
D:\230216_Lydo_Concerts\CSV\FASTACC-VN206-5022062-23021600-cut.csv
58.58996534347534
D:\230216_Lydo_Concerts\CSV\RESPACC-VN206-5022062-23021600-cut.csv
61.88442587852478
D:\230216_Lydo_Concerts\CSV\RR-VN206-5022062-23021600-cut.csv
62.08579635620117
63.003726959228516
[63.00472354888916, 2286.3696134090424]
D:\230216_Lydo_Concerts\CSV\DATA-VN203-5022057-23021600-cut.csv
0.030896663665771484
D:\230216_Lydo_Concerts\CSV\ECG-VN203-5022057-23021600-cut.csv
37.205010175704956
D:\230216_Lydo_Concerts\CSV\FASTACC-VN203-5022057-23021600-cut.csv
62.82039284706116
D:\230216_Lydo_Concerts\CSV\RESPACC-VN203-5022057-23021600-cut.csv
65.27833461761475
D:\230216_Lydo_Concerts\CSV\RR-VN203-5022057-23021600-cut.csv
65.41989374160767
66.33184742927551
[66.33184742927551, 2352.701460838

In [3]:
1715/66

29.3

In [82]:
dfiles = qex.qiosk_recordings(projectPath,projectName,seperator)
print(projectPath)
print(projectName)
dfiles

D:\230214_Equivital\
230214_Equivital


,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VA304,5022110,230214,0,DATA-VA304-5022110-23021400-cut.csv,csv,80491,2023-02-14 08:55:02+00:00,2023-02-14 12:59:47+00:00,...,[VA304],5022110.0,VA304,NaN,63,98.0,0.0,0.0,Stationary,Upright
1,DATA,BR604,5022198,230214,101,DATA-BR604-5022198-23021400_0-cut.csv,csv,80215,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[BR604],5022198.0,BR604,NaN,65,98.0,0.0,0.0,Stationary,Upright
2,DATA,VN205,5022060,230214,0,DATA-VN205-5022060-23021400-cut.csv,csv,80246,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[VN205],5022060.0,VN205,NaN,63,99.0,0.0,0.0,Stationary,Upright
3,DATA,PC702,5022240,230214,0,DATA-PC702-5022240-23021400-cut.csv,csv,80308,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[PC702],5022240.0,PC702,NaN,70,98.0,0.0,0.0,Stationary,Upright
4,DATA,VN202,5022051,230214,0,DATA-VN202-5022051-23021400-cut.csv,csv,80336,2023-02-14 08:55:04+00:00,2023-02-14 12:59:51+00:00,...,[VN202],5022051.0,VN202,NaN,84,99.0,0.0,0.0,Stationary,Upright
5,DATA,PC703,5022241,230214,0,DATA-PC703-5022241-23021400-cut.csv,csv,80996,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[PC703],5022241.0,PC703,NaN,74,100.0,0.0,0.0,Stationary,Upright
6,DATA,LS401,5022138,230214,0,DATA-LS401-5022138-23021400-cut.csv,csv,80302,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[LS401],5022138.0,LS401,NaN,70,99.0,0.0,0.0,Stationary,Upright
7,DATA,PC701,5022238,230214,0,DATA-PC701-5022238-23021400-cut.csv,csv,80391,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[PC701],5022238.0,PC701,NaN,80,98.0,0.0,0.0,Stationary,Upright
8,DATA,WW502,5022156,230214,0,DATA-WW502-5022156-23021400-cut.csv,csv,80378,2023-02-14 08:55:06+00:00,2023-02-14 12:59:51+00:00,...,[WW502],5022156.0,WW502,NaN,70,98.0,0.0,0.0,Stationary,Upright
9,DATA,LS403,5022141,230214,0,DATA-LS403-5022141-23021400-cut.csv,csv,80401,2023-02-14 08:55:07+00:00,2023-02-14 12:59:52+00:00,...,[LS403],5022141.0,LS403,NaN,88,98.0,0.0,0.0,Stationary,Upright


## Troubleshooting

In [69]:
shutil.move(fi,out_f)

'D:\\230214_Equivital\\SEM\\VN101\\23021400.SEM'

In [11]:
dfiles = qex.qiosk_recordings(discardPath, 'ToBeDiscarded',seperator)
print(len(dfiles))

585
